# Titanic

### Import Dependencies

In [1]:
# Set up TensorFlow
# For GPU users
%pip install tensorflow
%pip install kaggle
%pip install tensorflow_decision_forests

import tensorflow as tf
import pandas as pd
import numpy as np
import os

import tensorflow_decision_forests as tfdf
print(f"Found TF-DF {tfdf.__version__}")

# Verify the GPU setup
print(tf.config.list_physical_devices())

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2023-12-30 14:36:29.705990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 14:36:29.706072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 14:36:29.708500: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 14:36:29.724484: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 14:36:31.568287: W tensorflow/compiler/tf2

Found TF-DF 1.8.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-12-30 14:36:33.440922: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-30 14:36:33.500163: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-30 14:36:33.500396: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
if not os.path.exists("titanic/data/titanic.zip"):
  !kaggle competitions download -c titanic -p titanic/data
  !unzip titanic/data/titanic.zip -d titanic/data

In [3]:
!lspci | grep -i nvidia

01:00.0 3D controller: NVIDIA Corporation GP107M [GeForce GTX 1050 Mobile] (rev a1)


### Load Dataset

In [4]:
train_df = pd.read_csv("titanic/data/train.csv")
serving_df = pd.read_csv("titanic/data/test.csv")

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Prepare Dataset

In [5]:
# Preprocess data
def preprocess(df):
    """
    Preprocess data
    """
    df = df.copy()

    def normalize_name(x):
        """
        Normalize name
        """
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    def ticket_number(x):
        """
        Extract ticket number
        """
        return x.split(" ")[-1]

    def ticket_item(x):
        """
        Extract ticket item
        """
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    return df


preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)

preprocessed_train_df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C85,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C123,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,NaN,S,373450,NONE


In [6]:
# Create list of input features
input_features = list(preprocessed_train_df.columns)
input_features.remove("Ticket")
input_features.remove("PassengerId")
input_features.remove("Survived")

print(f"Input features: {input_features}")

Input features: ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked', 'Ticket_number', 'Ticket_item']


### Convert Pandas Dataset to Tensorflow Dataset

In [7]:
def tokenize_names(features, labels=None):
    """
    Divide the names into tokens. TF-DFcan consume text tokens natively.
    """
    features["Name"] = tf.strings.split(features["Name"])
    return features, labels


train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    preprocessed_train_df, label="Survived"
).map(tokenize_names)
serving_ds = tfdf.keras.pd_dataframe_to_tf_dataset(preprocessed_serving_df).map(
    tokenize_names
)

2023-12-30 14:36:33.884604: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-30 14:36:33.885162: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-30 14:36:33.885517: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### Train Model With Default Parameters

In [8]:
# Train model
model = tfdf.keras.GradientBoostedTreesModel(
    verbose=0,  # very few logs
    features=[tfdf.keras.FeatureUsage(name=n) for n in input_features],
    exclude_non_specified_features=True,  # Only use features in "features"
    random_seed=3,
)
model.fit(train_ds)

self_evaluation = model.make_inspector().evaluation()
print(f"Accuracy: {self_evaluation.accuracy} Loss: {self_evaluation.loss}")

[WARNING 23-12-30 14:36:34.6185 EST gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-30 14:36:34.6192 EST gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-30 14:36:34.6193 EST gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
2023-12-30 14:36:40.783312: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:504] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-12.2
  /usr/local/cuda
  /home/mattvanharn/repos/kaggle/titanic/titanic/lib/python3.11/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/mattvanharn/repos/kaggle/titanic/titanic/lib/python3.11/site-packages/te

Accuracy: 0.8229166865348816 Loss: 0.8424865007400513


### Make Predictions

In [9]:
def prediction_to_kaggle_format(model, threshold = 0.5):
  proba_survive = model.predict(serving_ds, verbose = 0)[:, 0]
  return pd.DataFrame({
      "PassengerId": serving_df["PassengerId"],
      "Survived": (proba_survive > threshold).astype(int)
  })

def make_submission(kaggle_predictions):
  path = "~/repos/kaggle/titanic/working/submission.csv"
  kaggle_predictions.to_csv(path, index = False)
  print(f"Submission saved to {path}")

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
!head ~/repos/kaggle/titanic/working/submission.csv

Submission saved to ~/repos/kaggle/titanic/working/submission.csv
PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,1
899,0
900,1


### Train Model With Hyperparameter Tuning

In [10]:
tuner = tfdf.tuner.RandomSearch(num_trials=1000)
tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

global_search_space = tuner.choice(
    "growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True
)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice(
    "sparse_oblique_normalization", ["NONE", "STANDARD_DEVIATION", "MIN_MAX"]
)
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])

# Tune the model. Notice the tuner = tuner
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=0)

tuned_self_evaluation = tuned_model.make_inspector().evaluation()
print(f"Accuracy: {tuned_self_evaluation.accuracy} Loss: {tuned_self_evaluation.loss}")

Use /tmp/tmp6bbennep as temporary training directory


[WARNING 23-12-30 14:36:43.7020 EST gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-30 14:36:43.7020 EST gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-12-30 14:36:43.7021 EST gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-12-30 14:38:24.0437 EST kernel.cc:1233] Loading model from path /tmp/tmp6bbennep/model/ with prefix f4e98927044f4247
[INFO 23-12-30 14:38:24.0742 EST decision_forest.cc:660] Model loaded with 30 root(s), 2236 node(s), and 12 input feature(s).
[INFO 23-12-30 14:38:24.0743 EST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 23-12-30 14:38:24.0743 EST kernel.cc:1061] Use fast generic engine


Accuracy: 0.8630136847496033 Loss: 0.6749962568283081
